In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# set the max columns to none
pd.set_option('display.max_columns', None)

import tensorflow as tf
from tensorflow import keras

## Data Hotel Goibibo 

In [40]:
df_goibibo = pd.read_csv('goibibo_com-travel_sample.csv')
# select potential parameter only
df_goibibo.drop(columns=['room_type','room_count','additional_info','province','area','locality','crawl_date','hotel_brand','hotel_category','pageurl','qts','query_time_stamp','room_area','similar_hotel','sitename','uniq_id','image_count'], inplace=True)
# drop data review
df_goibibo.drop(columns=['review_count_by_category','site_review_count','site_stay_review_rating','guest_recommendation'], inplace=True)

df_goibibo.head(3)

address     city country  \
0  15th Mile, N.H.21,Manali, District Kullu,Himac...   Manali   India   
1  A-585, Sushant Lok-1 ,Near Iffco Chowk Metro S...  Gurgaon   India   
2  Cobra Vaddo,Calungate Baga Road, Bardez, Calan...      Goa   India   

                                   hotel_description  \
0  The standard check-in time is 12:00 PM and the...   
1  The standard check-in time is 12:00 PM and the...   
2  The standard check-in time is 12:00 PM and the...   

                                    hotel_facilities  hotel_star_rating  \
0  Doctor on Call|Dry Cleaning|Laundry Service Av...                  2   
1  Airport Transfer Available / Surcharge|Banquet...                  0   
2  Swimming Pool|Bar / Lounge |Laundry Service Av...                  0   

    latitude  longitude                                  point_of_interest  \
0  32.139387  77.154660  Hadimba Temple|Naggar Village|Himalayan Nyingm...   
1  28.472097  77.072546  Sushant Lok|Sahara Mall|Amity International Sc...   
2  15.548398  73.757634  Anjuna Beach|Calangute Beach|Titos lane, baga|...   

        property_id        property_name property_type  \
0  HTLGBO1000016238     Baragarh Regency        Resort   
1  HTLGBO1000015822  Asian Suites A- 585   Guest House   
2  HTLGBO1000004551        Bevvan Resort        Resort   

                                     room_facilities  site_review_rating  \
0  Room Service |Basic Bathroom Amenities|Cable /...                 4.0   
1  Room Service |Air Conditioning |Basic Bathroom...                 4.5   
2  Room Service |Air Conditioning |Cable / Satell...                 2.5   

              state  
0  Himachal Pradesh  
1           Haryana  
2               Goa

In [41]:
# eksplorasi dataset dan drop null serta duplikat
print('jumlah property id: ',len(df_goibibo['property_id'].unique()))
print('jumlah property name: ',len(df_goibibo['property_name'].unique()))
print('jumlah property type: ',len(df_goibibo['property_type'].unique()))

df_goibibo.dropna(inplace=True)
df_goibibo.drop_duplicates(subset=['property_name'], inplace=True)
print('jumlah data setelah drop null dan duplicates: ',len(df_goibibo.dropna()))

jumlah property id:  4000
jumlah property name:  3805
jumlah property type:  18
jumlah data setelah drop null dan duplicates:  1953


In [42]:
# eksplorasi lanjutan
print('Daftar tipe properti: \n', len(df_goibibo['property_type'].unique()), df_goibibo['property_type'].unique())
print('Total kota: ',len(df_goibibo['city'].unique()))
print('Total provinsi: ',len(df_goibibo['state'].unique()), df_goibibo['state'].unique())
print('Daftar hotel rating: ',len(df_goibibo['hotel_star_rating'].unique()), df_goibibo['hotel_star_rating'].unique())

# room type terlalu beragam, tidak standar jenisnya
# print('Daftar Room Type: ',len(df_goibibo['room_type'].unique()))

Daftar tipe properti: 
 17 ['Resort' 'Guest House' 'Cottage' 'Hotel' 'Homestay' 'Palace'
 'Service Apartment' 'Hostel' 'Lodge' 'Tent' 'Villa' 'Houseboat'
 'Bungalow' 'BnB' 'Beach Hut' 'Motel' 'Farm Stay']
Total kota:  330
Total provinsi:  27 ['Himachal Pradesh' 'Haryana' 'Goa' 'Delhi' 'Rajasthan' 'Assam'
 'Tamil Nadu' 'Gujarat' 'West Bengal' 'Maharashtra' 'Karnataka' 'Kerala'
 'Punjab' 'Uttarakhand' 'Madhya Pradesh' 'Uttar Pradesh' 'Sikkim'
 'Andaman and Nicobar Islands' 'Andhra Pradesh' 'Orissa'
 'Jammu and Kashmir' 'Jharkhand' 'Pondicherry' 'Bihar' 'Meghalaya'
 'Dadra and Nagar Haveli' 'Chhattisgarh']
Daftar hotel rating:  6 [2 0 3 1 4 5]


In [43]:
# melihat tipe data
df_goibibo.dtypes

address                object
city                   object
country                object
hotel_description      object
hotel_facilities       object
hotel_star_rating       int64
latitude              float64
longitude             float64
point_of_interest      object
property_id            object
property_name          object
property_type          object
room_facilities        object
site_review_rating    float64
state                  object
dtype: object

In [44]:
# change nested data to array
df_goibibo['hotel_facilities'] = [item.split("|") for item in df_goibibo['hotel_facilities']]
df_goibibo['point_of_interest'] = [item.split("|") for item in df_goibibo['point_of_interest']]
df_goibibo['room_facilities'] = [item.split("|") for item in df_goibibo['room_facilities']]
df_goibibo

address          city country  \
0     15th Mile, N.H.21,Manali, District Kullu,Himac...        Manali   India   
1     A-585, Sushant Lok-1 ,Near Iffco Chowk Metro S...       Gurgaon   India   
2     Cobra Vaddo,Calungate Baga Road, Bardez, Calan...           Goa   India   
3                                                 Simsa        Manali   India   
4            8180 Street No.-6,Arakashan Road,Paharganj         Delhi   India   
...                                                 ...           ...     ...   
3992  sarvaritu vilas , inside udaipole , opp sagasj...       Udaipur   India   
3993                Near Yasholaxmi Nagar, Kolhapur Rd,  Ichalkaranji   India   
3994                     Near Railway Station , Kota Jn          Kota   India   
3997  Marol Maroshi Road, Near To Marol naka FireBri...        Mumbai   India   
3999                     N.H 6, A.Katapali, Near Gosala     Sambalpur   India   

                                      hotel_description  \
0     The standard check-in time is 12:00 PM and the...   
1     The standard check-in time is 12:00 PM and the...   
2     The standard check-in time is 12:00 PM and the...   
3     The standard check-in time is 12:00 PM and the...   
4     The standard check-in time is 12:00 PM and the...   
...                                                 ...   
3992  The standard check-in time is 12:00 PM and the...   
3993  The standard check-in time is 10:00 AM and the...   
3994  The standard check-in time is 12:00 PM and the...   
3997  The standard check-in time is 12:00 PM and the...   
3999  The standard check-in time is 10:00 AM and the...   

                                       hotel_facilities  hotel_star_rating  \
0     [Doctor on Call, Dry Cleaning, Laundry Service...                  2   
1     [Airport Transfer Available / Surcharge, Banqu...                  0   
2     [Swimming Pool, Bar / Lounge , Laundry Service...                  0   
3     [Doctor on Call, Dry Cleaning, Laundry Service...                  2   
4     [Internet Access - Surcharge, Laundry Service ...                  2   
...                                                 ...                ...   
3992  [Airport Transfer Available / Surcharge, Docto...                  2   
3993  [Bar / Lounge , Doctor on Call, Dry Cleaning, ...                  2   
3994  [Bar / Lounge , Doctor on Call, Laundry Servic...                  2   
3997  [Banquet Facilities, Bar / Lounge , Currency E...                  3   
3999  [Business Center , Doctor on Call, Parking Fac...                  2   

       latitude  longitude                                  point_of_interest  \
0     32.139387  77.154660  [Hadimba Temple, Naggar Village, Himalayan Nyi...   
1     28.472097  77.072546  [Sushant Lok, Sahara Mall, Amity International...   
2     15.548398  73.757634  [Anjuna Beach, Calangute Beach, Titos lane, ba...   
3     32.223603  77.185900  [The Mall, Tibetian Monastery, Vashisht Hot Ba...   
4     28.646777  77.212735  [Gaffar Market, YMCA Institute, Agrasen Boali,...   
...         ...        ...                                                ...   
3992  24.575598  73.696959  [Jagdish Temple, City Palace, Udaipur Bus Stan...   
3993  16.690042  74.456394                                 [Yasholaxmi Nagar]   
3994  25.222694  75.878170                  [Kota Airport, Station Road Area]   
3997  19.110926  72.879038  [Mount Mary Church, Juhu Beach, Domestic Airpo...   
3999  21.448978  83.912502                                         [Katapali]   

           property_id                           property_name property_type  \
0     HTLGBO1000016238                        Baragarh Regency        Resort   
1     HTLGBO1000015822                     Asian Suites A- 585   Guest House   
2     HTLGBO1000004551                           Bevvan Resort        Resort   
3     HTLGBO1000007993                       Apple Inn Cottage       Cottage   
4     HTLGBO1000008281                     Anmol Hotel P

In [45]:
df_goibibo[['hotel_facilities','room_facilities','point_of_interest']]

hotel_facilities  \
0     [Doctor on Call, Dry Cleaning, Laundry Service...   
1     [Airport Transfer Available / Surcharge, Banqu...   
2     [Swimming Pool, Bar / Lounge , Laundry Service...   
3     [Doctor on Call, Dry Cleaning, Laundry Service...   
4     [Internet Access - Surcharge, Laundry Service ...   
...                                                 ...   
3992  [Airport Transfer Available / Surcharge, Docto...   
3993  [Bar / Lounge , Doctor on Call, Dry Cleaning, ...   
3994  [Bar / Lounge , Doctor on Call, Laundry Servic...   
3997  [Banquet Facilities, Bar / Lounge , Currency E...   
3999  [Business Center , Doctor on Call, Parking Fac...   

                                        room_facilities  \
0     [Room Service , Basic Bathroom Amenities, Cabl...   
1     [Room Service , Air Conditioning , Basic Bathr...   
2     [Room Service , Air Conditioning , Cable / Sat...   
3     [Basic Bathroom Amenities, Cable / Satellite /...   
4     [Basic Bathroom Amenities, Cable / Satellite /...   
...                                                 ...   
3992  [Room Service , Basic Bathroom Amenities, Cabl...   
3993  [Room Service , Air Conditioning , Alarm Clock...   
3994  [Room Service , Air Conditioning , Basic Bathr...   
3997  [Air Conditioning , Cable / Satellite / Pay TV...   
3999  [Room Service , Basic Bathroom Amenities,  Hot...   

                                      point_of_interest  
0     [Hadimba Temple, Naggar Village, Himalayan Nyi...  
1     [Sushant Lok, Sahara Mall, Amity International...  
2     [Anjuna Beach, Calangute Beach, Titos lane, ba...  
3     [The Mall, Tibetian Monastery, Vashisht Hot Ba...  
4     [Gaffar Market, YMCA Institute, Agrasen Boali,...  
...                                                 ...  
3992  [Jagdish Temple, City Palace, Udaipur Bus Stan...  
3993                                 [Yasholaxmi Nagar]  
3994                  [Kota Airport, Station Road Area]  
3997  [Mount Mary Church, Juhu Beach, Domestic Airpo...  
3999                                         [Katapali]  

[1953 rows x 3 columns]

In [46]:
# rearrange the feature position
df_goibibo = df_goibibo[['property_id','property_name','hotel_description','address','point_of_interest','city','state','country','latitude','longitude','property_type','hotel_star_rating','hotel_facilities','room_facilities','site_review_rating']]

# recreate the property_id for better indexing
hid = []
for n in range(1, len(df_goibibo)+1):
    hid.append('H{0:06}'.format(n))
df_goibibo['property_id'] = hid
# save data hotel for non ML
# df_goibibo.to_csv('DataHotel_nonML.csv')

In [47]:
df_goibibo.head(3)

property_id        property_name  \
0     H000001     Baragarh Regency   
1     H000002  Asian Suites A- 585   
2     H000003        Bevvan Resort   

                                   hotel_description  \
0  The standard check-in time is 12:00 PM and the...   
1  The standard check-in time is 12:00 PM and the...   
2  The standard check-in time is 12:00 PM and the...   

                                             address  \
0  15th Mile, N.H.21,Manali, District Kullu,Himac...   
1  A-585, Sushant Lok-1 ,Near Iffco Chowk Metro S...   
2  Cobra Vaddo,Calungate Baga Road, Bardez, Calan...   

                                   point_of_interest     city  \
0  [Hadimba Temple, Naggar Village, Himalayan Nyi...   Manali   
1  [Sushant Lok, Sahara Mall, Amity International...  Gurgaon   
2  [Anjuna Beach, Calangute Beach, Titos lane, ba...      Goa   

              state country   latitude  longitude property_type  \
0  Himachal Pradesh   India  32.139387  77.154660        Resort   
1           Haryana   India  28.472097  77.072546   Guest House   
2               Goa   India  15.548398  73.757634        Resort   

   hotel_star_rating                                   hotel_facilities  \
0                  2  [Doctor on Call, Dry Cleaning, Laundry Service...   
1                  0  [Airport Transfer Available / Surcharge, Banqu...   
2                  0  [Swimming Pool, Bar / Lounge , Laundry Service...   

                                     room_facilities  site_review_rating  
0  [Room Service , Basic Bathroom Amenities, Cabl...                 4.0  
1  [Room Service , Air Conditioning , Basic Bathr...                 4.5  
2  [Room Service , Air Conditioning , Cable / Sat...                 2.5

In [48]:
hotel = set()
for item in df_goibibo['hotel_facilities']:
    for h in item:
        hotel.add(h)
room = set()
for item in df_goibibo['room_facilities']:
    for r in item:
        room.add(r)
poi = set()
for item in df_goibibo['point_of_interest']:
    for p in item:
        poi.add(p)

In [49]:
print('Jumlah unique hotel facilities: ', len(hotel))
print('Jumlah unique room facilities: ', len(room))
print('Jumlah unique point of interest: ', len(poi))
# POI Terlalu beragam dan tidak standar sehingga tidak diperhitungkan dahulu dalam ML

Jumlah unique hotel facilities:  202
Jumlah unique room facilities:  166
Jumlah unique point of interest:  4218


In [50]:
from sklearn.preprocessing import MultiLabelBinarizer
# one-hot encoding hotel facilities
mlb = MultiLabelBinarizer()
x = mlb.fit_transform(df_goibibo['hotel_facilities'])
# one-hot encoding room facilities
mlb2 = MultiLabelBinarizer()
y = mlb2.fit_transform(df_goibibo['room_facilities'])

In [51]:
df_hotel_facilities = pd.DataFrame(x, columns=mlb.classes_)
df_room_facilities = pd.DataFrame(y, columns=mlb2.classes_)

In [52]:
df_hotel_facilities.head(3)

Bike on Rent   Cook Services   Drivers Rest Room  24 Hour Front Desk  \
0              0               1                   0                   0   
1              0               0                   0                   1   
2              0               0                   0                   0   

   ATM / Banking  Access Via Exterior Corridors  Adjoining Rooms  \
0              0                              0                0   
1              0                              0                0   
2              0                              0                0   

   Air / Rail Booking  Air-Conditioning - Central  \
0                   1                           0   
1                   0                           0   
2                   0                           0   

   Airport Transfer Available / Surcharge  Airport Transfer Free  \
0                                       0                      0   
1                                       1                      0   
2                                       0                      0   

   Amphitheatre  Anti-Slip Ramps  Audio - Visual Equipment  \
0             0                0                         0   
1             0                0                         0   
2             0                0                         0   

   Ayurvedic Facilities  Ayurvedic Facilities   \
0                     0                      0   
1                     0                      0   
2                     0                      0   

   Babysitting/ Child Care (surcharge)  Badminttion Court  Baggage Room  \
0                                    0                  0             0   
1                                    0                  0             0   
2                                    0                  0             0   

   Ballroom  Banquet Facilities  Bar / Lounge   Barbecue Area  \
0         0                   0              0              0   
1         0                   1              0              0   
2         0                   0              1              0   

   Basketball Court   Beauty Salon - on charge  Billiards  Board Room  \
0                  0                         0          0           0   
1                  0                         0          0           0   
2                  0                         0          0           0   

   Boating  Bowling  Business Center  Business Center   CCTV  Camel Ride  \
0        0        0                0                 0     0           0   
1        0        0                0                 1     0           0   
2        0        0                0                 0     0           0   

   Campfire / Bon Fire  Car Rental  Casino  Catering  Cell Phone Rental  \
0                    0           1       0         1                  0   
1                    0           0       0         0                  0   
2                    0           0       0         0                  0   

   Children's Park  City Shuttle Service  Clubhouse  Coffee Shop / Cafe  \
0                1                     0          0                   0   
1                0                     0          0                   0   
2                0                     0          0                   0   

   Computer Rental  Concierge  Conference Facility  Conference Facility   \
0                0          0                    0                     0   
1                0          0                    0                     0   
2                0          0                    0                     0   

   Courier Service  Courtesy Coach  Currency Exchange  \
0                1               0                  0   
1                0               0                  1   
2                0               0                  0   

   Dance Performances (on demand)  Disabled Friendly Facilities   \
0                               1                              0   
1                               0                         

In [53]:
print(len(df_hotel_facilities.columns.values))
list(df_hotel_facilities.columns.values)

202


[' Bike on Rent',
 ' Cook Services',
 ' Drivers Rest Room',
 '24 Hour Front Desk',
 'ATM / Banking',
 'Access Via Exterior Corridors',
 'Adjoining Rooms',
 'Air / Rail Booking',
 'Air-Conditioning - Central',
 'Airport Transfer Available / Surcharge',
 'Airport Transfer Free',
 'Amphitheatre',
 'Anti-Slip Ramps',
 'Audio - Visual Equipment',
 'Ayurvedic Facilities',
 'Ayurvedic Facilities ',
 'Babysitting/ Child Care (surcharge)',
 'Badminttion Court',
 'Baggage Room',
 'Ballroom',
 'Banquet Facilities',
 'Bar / Lounge ',
 'Barbecue Area',
 'Basketball Court ',
 'Beauty Salon - on charge',
 'Billiards',
 'Board Room',
 'Boating',
 'Bowling',
 'Business Center',
 'Business Center ',
 'CCTV',
 'Camel Ride',
 'Campfire / Bon Fire',
 'Car Rental',
 'Casino',
 'Catering',
 'Cell Phone Rental',
 "Children's Park",
 'City Shuttle Service',
 'Clubhouse',
 'Coffee Shop / Cafe',
 'Computer Rental',
 'Concierge',
 'Conference Facility',
 'Conference Facility ',
 'Courier Service',
 'Courtesy Coac

In [54]:
df_room_facilities.head(3)

Hot / Cold Running Water  120 AC  220 AC  Air Conditioning  \
0                          1       0       0                 0   
1                          1       0       0                 0   
2                          1       0       0                 0   

   Air Conditioning   Air Cooler  Alarm Clock  Attached Bathroom  Balcony  \
0                  0           0            0                  1        0   
1                  1           0            0                  1        0   
2                  1           0            0                  1        0   

   Balcony / Terrace  Basic Bathroom Amenities  Bath Tub  Bathrobe  \
0                  0                         1         0         0   
1                  0                         1         0         0   
2                  0                         0         0         0   

   Bathrobe (on request)   Bathrobes  Bathroom Amenities  Bathtub  \
0                       0          1                   0        0   
1                       0          0                   0        0   
2                       0          0                   0        0   

   Bathtub Only  Bedside Table  Bidet  Blackout Drapes  Breakfast Available  \
0             0              1      0                0                    0   
1             0              0      0                0                    1   
2             0              0      0                0                    0   

   Breakfast Available (surcharge)  Breakfast chargeable  \
0                                1                     1   
1                                1                     0   
2                                0                     0   

   Cable / Satellite / Pay TV available  Cable/Satellite TV  Ceiling Fan  \
0                                     1                   0            1   
1                                     1                   0            1   
2                                     1                   0            1   

   Centrally Heated  Channel Music  Citrus Juicer  Climate Control  \
0                 0              0              0                0   
1                 0              0              0                0   
2                 0              0              0                0   

   Clock Radio  Closet  Clothes Dryer  Clothes Hook in Bathroom  \
0            0       0              0                         1   
1            0       0              0                         0   
2            0       0              0                         0   

   Clothes Press  Coffee Table  Color TV  Complete Bedding  \
0              0             0         0                 1   
1              0             1         0                 0   
2              0             0         0                 0   

   Complimentary Use of Bicycle  Computer  Cribs Available  \
0                             0         0                0   
1                             0         0                0   
2                             0         0                0   

   Crockery / Cutlery  Curtains/Shades  Custom Duvets / Linen  \
0                   0                0                      0   
1                   0                0                      0   
2                   0                0                      0   

   Daily Room Cleaning  Desk  Desk in Room  Direct Dial Phone  \
0                    1     0             1                  0   
1                    0     0             0                  0   
2                    0     0             1                  0   

   Disabled Friendly Bathrooms  Do-Not-Disturb Notice  Dressing Area  \
0                            0                      0              0   
1                            0                      0              0   
2                            0                      0              0   

   Dry Cleaning  Dustbins  Electronic Weighing Machine  \
0             0         1                            0   
1             0         0         

In [55]:
print(len(df_room_facilities.columns.values))
list(df_room_facilities.columns.values)

166


[' Hot / Cold Running Water',
 '120 AC',
 '220 AC',
 'Air Conditioning',
 'Air Conditioning ',
 'Air Cooler',
 'Alarm Clock',
 'Attached Bathroom',
 'Balcony',
 'Balcony / Terrace',
 'Basic Bathroom Amenities',
 'Bath Tub',
 'Bathrobe',
 'Bathrobe (on request) ',
 'Bathrobes',
 'Bathroom Amenities',
 'Bathtub',
 'Bathtub Only',
 'Bedside Table',
 'Bidet',
 'Blackout Drapes',
 'Breakfast Available',
 'Breakfast Available (surcharge)',
 'Breakfast chargeable',
 'Cable / Satellite / Pay TV available',
 'Cable/Satellite TV',
 'Ceiling Fan',
 'Centrally Heated',
 'Channel Music',
 'Citrus Juicer',
 'Climate Control',
 'Clock Radio',
 'Closet',
 'Clothes Dryer',
 'Clothes Hook in Bathroom',
 'Clothes Press',
 'Coffee Table',
 'Color TV',
 'Complete Bedding',
 'Complimentary Use of Bicycle',
 'Computer',
 'Cribs Available',
 'Crockery / Cutlery',
 'Curtains/Shades',
 'Custom Duvets / Linen',
 'Daily Room Cleaning',
 'Desk',
 'Desk in Room',
 'Direct Dial Phone',
 'Disabled Friendly Bathrooms'

In [56]:
# merge dataframe utama dengan hotel facilities
df_final = pd.concat([df_goibibo.reset_index(drop=True),df_hotel_facilities.reset_index(drop=True)], axis=1)
print(df_final.shape)
df_final.head(3)

(1953, 217)


property_id        property_name  \
0     H000001     Baragarh Regency   
1     H000002  Asian Suites A- 585   
2     H000003        Bevvan Resort   

                                   hotel_description  \
0  The standard check-in time is 12:00 PM and the...   
1  The standard check-in time is 12:00 PM and the...   
2  The standard check-in time is 12:00 PM and the...   

                                             address  \
0  15th Mile, N.H.21,Manali, District Kullu,Himac...   
1  A-585, Sushant Lok-1 ,Near Iffco Chowk Metro S...   
2  Cobra Vaddo,Calungate Baga Road, Bardez, Calan...   

                                   point_of_interest     city  \
0  [Hadimba Temple, Naggar Village, Himalayan Nyi...   Manali   
1  [Sushant Lok, Sahara Mall, Amity International...  Gurgaon   
2  [Anjuna Beach, Calangute Beach, Titos lane, ba...      Goa   

              state country   latitude  longitude property_type  \
0  Himachal Pradesh   India  32.139387  77.154660        Resort   
1           Haryana   India  28.472097  77.072546   Guest House   
2               Goa   India  15.548398  73.757634        Resort   

   hotel_star_rating                                   hotel_facilities  \
0                  2  [Doctor on Call, Dry Cleaning, Laundry Service...   
1                  0  [Airport Transfer Available / Surcharge, Banqu...   
2                  0  [Swimming Pool, Bar / Lounge , Laundry Service...   

                                     room_facilities  site_review_rating  \
0  [Room Service , Basic Bathroom Amenities, Cabl...                 4.0   
1  [Room Service , Air Conditioning , Basic Bathr...                 4.5   
2  [Room Service , Air Conditioning , Cable / Sat...                 2.5   

    Bike on Rent   Cook Services   Drivers Rest Room  24 Hour Front Desk  \
0              0               1                   0                   0   
1              0               0                   0                   1   
2              0               0                   0                   0   

   ATM / Banking  Access Via Exterior Corridors  Adjoining Rooms  \
0              0                              0                0   
1              0                              0                0   
2              0                              0                0   

   Air / Rail Booking  Air-Conditioning - Central  \
0                   1                           0   
1                   0                           0   
2                   0                           0   

   Airport Transfer Available / Surcharge  Airport Transfer Free  \
0                                       0                      0   
1                                       1                      0   
2                                       0                      0   

   Amphitheatre  Anti-Slip Ramps  Audio - Visual Equipment  \
0             0                0                         0   
1             0                0                         0   
2             0                0                         0   

   Ayurvedic Facilities  Ayurvedic Facilities   \
0                     0                      0   
1                     0                      0   
2                     0                      0   

   Babysitting/ Child Care (surcharge)  Badminttion Court  Baggage Room  \
0                                    0                  0             0   
1                                    0                  0             0   
2                                    0                  0             0   

   Ballroom  Banquet Facilities  Bar / Lounge   Barbecue Area  \
0         0                   0              0              0   
1         0                   1              0              0   
2         0                   0              1              0   

   Basketball Court   Beauty Salon - on charge  Billiards  Board Room  \
0                  0                         0          0           0   
1                  0                     

In [57]:
# merge dataframe utama+hotel facilities dengan room facilities
df_final = pd.concat([df_final.reset_index(drop=True),df_room_facilities.reset_index(drop=True)], axis=1)
print(df_final.shape)
df_final.head(3)

(1953, 383)


property_id        property_name  \
0     H000001     Baragarh Regency   
1     H000002  Asian Suites A- 585   
2     H000003        Bevvan Resort   

                                   hotel_description  \
0  The standard check-in time is 12:00 PM and the...   
1  The standard check-in time is 12:00 PM and the...   
2  The standard check-in time is 12:00 PM and the...   

                                             address  \
0  15th Mile, N.H.21,Manali, District Kullu,Himac...   
1  A-585, Sushant Lok-1 ,Near Iffco Chowk Metro S...   
2  Cobra Vaddo,Calungate Baga Road, Bardez, Calan...   

                                   point_of_interest     city  \
0  [Hadimba Temple, Naggar Village, Himalayan Nyi...   Manali   
1  [Sushant Lok, Sahara Mall, Amity International...  Gurgaon   
2  [Anjuna Beach, Calangute Beach, Titos lane, ba...      Goa   

              state country   latitude  longitude property_type  \
0  Himachal Pradesh   India  32.139387  77.154660        Resort   
1           Haryana   India  28.472097  77.072546   Guest House   
2               Goa   India  15.548398  73.757634        Resort   

   hotel_star_rating                                   hotel_facilities  \
0                  2  [Doctor on Call, Dry Cleaning, Laundry Service...   
1                  0  [Airport Transfer Available / Surcharge, Banqu...   
2                  0  [Swimming Pool, Bar / Lounge , Laundry Service...   

                                     room_facilities  site_review_rating  \
0  [Room Service , Basic Bathroom Amenities, Cabl...                 4.0   
1  [Room Service , Air Conditioning , Basic Bathr...                 4.5   
2  [Room Service , Air Conditioning , Cable / Sat...                 2.5   

    Bike on Rent   Cook Services   Drivers Rest Room  24 Hour Front Desk  \
0              0               1                   0                   0   
1              0               0                   0                   1   
2              0               0                   0                   0   

   ATM / Banking  Access Via Exterior Corridors  Adjoining Rooms  \
0              0                              0                0   
1              0                              0                0   
2              0                              0                0   

   Air / Rail Booking  Air-Conditioning - Central  \
0                   1                           0   
1                   0                           0   
2                   0                           0   

   Airport Transfer Available / Surcharge  Airport Transfer Free  \
0                                       0                      0   
1                                       1                      0   
2                                       0                      0   

   Amphitheatre  Anti-Slip Ramps  Audio - Visual Equipment  \
0             0                0                         0   
1             0                0                         0   
2             0                0                         0   

   Ayurvedic Facilities  Ayurvedic Facilities   \
0                     0                      0   
1                     0                      0   
2                     0                      0   

   Babysitting/ Child Care (surcharge)  Badminttion Court  Baggage Room  \
0                                    0                  0             0   
1                                    0                  0             0   
2                                    0                  0             0   

   Ballroom  Banquet Facilities  Bar / Lounge   Barbecue Area  \
0         0                   0              0              0   
1         0                   1              0              0   
2         0                   0              1              0   

   Basketball Court   Beauty Salon - on charge  Billiards  Board Room  \
0                  0                         0          0           0   
1                  0                     

In [58]:
# save data hotel for ML
# df_final.to_csv('DataHotel_ML.csv')

## Further Filtering Hotel Features

In [59]:
# sheets dapat dilihat di https://docs.google.com/spreadsheets/d/16DLw67DokkkhlyDhjD1I98R9JbUOqppULbB8ALC4h7U/edit?pli=1#gid=1289123885
df_hotel = pd.read_csv('https://raw.githubusercontent.com/zulfauzi92/Hotel_Recomendation_Model_Traveloka/main/Eksplorasi%20Data/Main%20Dataset/DataHotel_ML.csv', index_col=[0])
df_hotel

property_id                                      property_name  \
0        H000001  Homewood Suites by Hilton RaleighDurham APRese...   
1        H000002  Four Points by Sheraton Orlando International ...   
2        H000003                Hampton Inn Pittsburgh/West Mifflin   
3        H000004                       JW Marriott Atlanta Buckhead   
4        H000005                               Best Western-rambler   
...          ...                                                ...   
1306     H001307                                      Cupertino Inn   
1307     H001308               Best Western Plus-concordville Hotel   
1308     H001309               Georgia Tech Hotel Conference Center   
1309     H001310          Best Western Plus Nashville Airport Hotel   
1310     H001311  Best Western Plus Dallas Hotel & Conference Ce...   

                                      hotel_description  \
0     The standard check-in time is 12:00 PM and the...   
1     The standard check-in time is 12:00 PM and the...   
2     The standard check-in time is 12:00 PM and the...   
3     The standard check-in time is 12:00 PM and the...   
4     The standard check-in time is 02:00 PM and the...   
...                                                 ...   
1306  The standard check-in time is 12:00 PM and the...   
1307  The standard check-in time is 12:00 PM and the...   
1308  The standard check-in time is 12:00 PM and the...   
1309  The standard check-in time is 10:00 AM and the...   
1310  The standard check-in time is 12:00 PM and the...   

                        address  \
0          4603 Central Park Dr   
1      5905 International Drive   
2      1550 Lebanon Church Road   
3            3300 Lenox Road NE   
4          457 US Highway 85 87   
...                         ...   
1306       10889 N De Anza Blvd   
1307          675 Conchester Rd   
1308           800 Spring St Nw   
1309      911 Airport Center Dr   
1310  8051 Lyndon B Johnson Fwy   

                                      point_of_interest        city state  \
0     ['Hadimba Temple', 'Naggar Village', 'Himalaya...      Durham    NC   
1     ['Sushant Lok', 'Sahara Mall', 'Amity Internat...     Orlando    FL   
2     ['The Mall', 'Tibetian Monastery', 'Vashisht H...  Pittsburgh    PA   
3     ['International Airport', 'Domestic Airport', ...     Atlanta    GA   
4     ['Others', 'K.L. Saini Stadium', 'BAPS Shri Sw...  Walsenburg    CO   
...                                                 ...         ...   ...   
1306                          ['Maharana Pratap Nagar']   Cupertino    CA   
1307  ['Gaffar Market', 'Gurudwara Rakabganj Sahib',...  Glen Mills    PA   
1308  ['The Naag (cobra) temple', 'Madhatop', 'Kud',...     Atlanta    GA   
1309                               ['Yasholaxmi Nagar']   Nashville    TN   
1310              ['Kota Airport', 'Station Road Area']      Dallas    TX   

     country  latitude  longitude property_type  hotel_star_rating  \
0         US      35.9      -78.8        Resort                  2   
1         US      28.5      -81.5   Guest House                  0   
2         US      40.3      -80.0       Cottage                  2   
3         US      33.8      -84.4         Hotel                  3   
4         US      37.7     -104.8      Homestay                  1   
...      ...       ...        ...           ...                ...   
1306      US      37.3     -122.0         Hotel                  2   
1307      US      39.9      -75.5         Hotel                  3   
1308      US      33.8      -84.4   Guest House                  0   
1309      US      36.1      -86.7         Hotel                  2   
1310      US      32.9      -96.8         Hotel                  2   

                                       hotel_facilities  \
0     ['Doctor on Call', 'Dry Cleaning', 'Laundry Se...   
1     ['Airport Transfer Available / Surcharge', 'Ba...   
2     ['Doctor on Call', 'Dry Cleaning', 'Laundry Se...   
3     ['Banquet Faci

### Merge Hotel Feature (merah)

In [60]:
# Membuat list feature yang akan di drop
feature_drop = []

In [61]:
# melakukan assign ke salah satu fitur dan sisanya di masukan ke list feature yang akan di drop nanti
# selalu perhatikan spasi di feature
df_hotel['Air Conditioning'] = df_hotel['Air Conditioning'] + df_hotel['Air Conditioning ']
feature_drop.append('Air Conditioning ')

df_hotel['Balcony'] = df_hotel['Balcony'] + df_hotel['Balcony / Terrace']
feature_drop.append('Balcony / Terrace')

df_hotel['Hot / Cold Running Water'] = df_hotel[' Hot / Cold Running Water'] + df_hotel['Hot / Cold Running Water']
feature_drop.append(' Hot / Cold Running Water')

df_hotel['Bathroom Amenities'] = df_hotel['Basic Bathroom Amenities'] + df_hotel['Bathroom Amenities']
feature_drop.append('Basic Bathroom Amenities')

df_hotel['Bathtub'] = df_hotel['Bath Tub'] + df_hotel['Bathtub']
feature_drop.append('Bath Tub')

df_hotel['Bathrobe'] = df_hotel['Bathrobe'] + df_hotel['Bathrobe (on request) ']
feature_drop.append('Bathrobe (on request) ')

df_hotel['Breakfast Available'] = df_hotel['Breakfast Available'] + df_hotel['Breakfast Available (surcharge)']
feature_drop.append('Breakfast Available (surcharge)')

df_hotel['Television'] = df_hotel['Cable / Satellite / Pay TV available'] + df_hotel['Cable/Satellite TV'] + df_hotel['TV'] + df_hotel['Television']
feature_drop.append('Cable / Satellite / Pay TV available')
feature_drop.append('Cable/Satellite TV')
feature_drop.append('TV')

df_hotel['Desk'] = df_hotel['Desk'] + df_hotel['Desk in Room']
feature_drop.append('Desk in Room')

df_hotel['Newspapers'] = df_hotel['Free Newspaper'] + df_hotel['Newspapers'] + df_hotel['Newspapers In Lobby']
feature_drop.append('Free Newspaper')
feature_drop.append('Newspapers In Lobby')

df_hotel['HD / LCD Satellite TV'] = df_hotel['HD / LCD Satellite TV'] + df_hotel['HD / LCD Satellite TV ']
feature_drop.append('HD / LCD Satellite TV ')

df_hotel['Hairdryer'] = df_hotel['Hairdryer'] + df_hotel['Hairdryer (on request)']
feature_drop.append('Hairdryer (on request)')

df_hotel['Heating Facility on Request'] = df_hotel['Heating Facility (on request)'] + df_hotel['Heating Facility (on request) '] + df_hotel['Room Heater']
feature_drop.append('Heating Facility (on request)')
feature_drop.append('Heating Facility (on request) ')
feature_drop.append('Room Heater')

df_hotel['Safe'] = df_hotel['In-Room Safe'] + df_hotel['Safe (on request)'] + df_hotel['Safe - In - Room'] + df_hotel['Safe - In - Room ']
feature_drop.append('In-Room Safe')
feature_drop.append('Safe - In - Room')
feature_drop.append('Safe - In - Room ')

df_hotel['Internet Access'] = df_hotel['Internet Access'] + df_hotel['Internet Connection In Room'] + df_hotel['Free Internet Access'] + df_hotel['Internet / Fax (Reception area only)'] + df_hotel['Internet Access '] + df_hotel['Internet Access - Surcharge'] + df_hotel['Internet Access Free'] + df_hotel['Internet Access.1'] 
feature_drop.append('Internet Access')
feature_drop.append('Internet Access ')
feature_drop.append('Internet Access Free')
feature_drop.append('Internet Connection In Room')
feature_drop.append('Free Internet Access')
feature_drop.append('Internet / Fax (Reception area only)')
feature_drop.append('Internet Access.1')


df_hotel['Iron / Ironing board'] = df_hotel['Iron / Ironing board available'] + df_hotel['Iron/Ironing Board']
feature_drop.append('Iron / Ironing board available')
feature_drop.append('Iron/Ironing Board')

df_hotel['Mini Bar'] = df_hotel['Mini Bar'] + df_hotel['Mini bar - On Charge ']
feature_drop.append('Mini bar - On Charge ')

df_hotel['Room Service'] = df_hotel['Room Service'] + df_hotel['Room Service '] + df_hotel['Room Service (24 Hours)'] + df_hotel['Room Service .1'] + df_hotel['Room Service.1']
feature_drop.append('Room Service')
feature_drop.append('Room Service ')
feature_drop.append('Room Service (24 Hours)')
feature_drop.append('Room Service .1')
feature_drop.append('Room Service.1')

df_hotel['Shower'] = df_hotel['Shower'] + df_hotel['Shower Facility Available']
feature_drop.append('Shower Facility Available')

df_hotel['Shower Cap'] = df_hotel['Shower Cap'] + df_hotel['Shower Caps Available']
feature_drop.append('Shower Caps Available')

df_hotel['Sofa Table / Extra Seats'] = df_hotel['Sofa Table / Extra Seats'] + df_hotel['Sofa Table / Extra Seats ']
feature_drop.append('Sofa Table / Extra Seats ')

df_hotel['Telephone'] = df_hotel['Telephone'] + df_hotel['Two Line Telephone'] + df_hotel['Working Desk Phone']
feature_drop.append('Two Line Telephone')
feature_drop.append('Working Desk Phone')

df_hotel['Microwave / Oven'] = df_hotel['Microwave Oven Available '] + df_hotel['Stove / Oven']
feature_drop.append('Microwave Oven Available ')
feature_drop.append('Stove / Oven')

df_hotel['Refrigerator'] = df_hotel['Mini Refrigerator'] + df_hotel['Refrigerator']
feature_drop.append('Refrigerator')

df_hotel['Attached Bathroom'] = df_hotel['Attached Bathroom'] + df_hotel['Rooms with Attached Bathrooms']
feature_drop.append('Rooms with Attached Bathrooms')

df_hotel['Window Open / Ventilation'] = df_hotel['Room with Window/Ventilation'] + df_hotel['Window Opens']
feature_drop.append('Room with Window/Ventilation')
feature_drop.append('Window Opens')

df_hotel['Front Desk'] = df_hotel['24 Hour Front Desk'] + df_hotel['Front Desk']
feature_drop.append('24 Hour Front Desk')

df_hotel['Business Center'] = df_hotel['Business Center'] + df_hotel['Business Center ']
feature_drop.append('Business Center ')

df_hotel['Conference Facility'] = df_hotel['Conference Facility'] + df_hotel['Conference Facility ']
feature_drop.append('Conference Facility ')

df_hotel['Karaoke'] = df_hotel['Karaoke'] + df_hotel['Karaoke ']
feature_drop.append('Karaoke ')

df_hotel['Massage Services'] = df_hotel['Massage Services'] + df_hotel['Massage Services ']
feature_drop.append('Massage Services ')

df_hotel['Multi Cuisine Restaurant'] = df_hotel['Multi Cuisine Restaurant'] + df_hotel['Multi Cuisine Restaurant ']
feature_drop.append('Multi Cuisine Restaurant ')

df_hotel['No Smoking Room'] = df_hotel['Non Smoking Hotel'] + df_hotel['Non Smoking Rooms'] + df_hotel['No Smoking Room']
feature_drop.append('Non Smoking Hotel')
feature_drop.append('Non Smoking Rooms')

df_hotel['Open Air Restaurant / Dining'] = df_hotel['Open Air Restaurant / Dining'] + df_hotel['Open Air Restaurant / Dining ']
feature_drop.append('Open Air Restaurant / Dining ')

df_hotel['Outdoor Parking'] = df_hotel['Outdoor Parking - Non Secured'] + df_hotel['Outdoor Parking - Secured']
feature_drop.append('Outdoor Parking - Non Secured')
feature_drop.append('Outdoor Parking - Secured')

df_hotel['Parking Facility'] = df_hotel['Parking (Surcharge)'] + df_hotel['Parking Facilities Available'] + df_hotel['Parking Facility']
feature_drop.append('Parking (Surcharge)')
feature_drop.append('Parking Facilities Available')

df_hotel['Porter Services'] = df_hotel['Porter / Bellhop'] + df_hotel['Porter Services Available']
feature_drop.append('Porter / Bellhop')
feature_drop.append('Porter Services Available')

df_hotel['Smoking Area'] = df_hotel['Smoking Area'] + df_hotel['Smoking Area '] + df_hotel['Smoking Rooms']
feature_drop.append('Smoking Area ')
feature_drop.append('Smoking Rooms')

df_hotel['Spa Services']  = df_hotel['Spa On Site - Chargeable'] + df_hotel['Spa Services Nearby'] + df_hotel['Spa Services Onsite - Free']
feature_drop.append('Spa On Site - Chargeable')
feature_drop.append('Spa Services Nearby')
feature_drop.append('Spa Services Onsite - Free')

df_hotel['Speciality Restaurant'] = df_hotel['Speciality Restaurant'] + df_hotel['Speciality Restaurant ']
feature_drop.append('Speciality Restaurant ')

df_hotel['Multi Lingual Staff'] = df_hotel['Multi Lingual Staff'] + df_hotel['Translation Services']
feature_drop.append('Translation Services')

df_hotel['Baggage Room'] = df_hotel['Baggage Room'] + df_hotel['Luggage Storage']
feature_drop.append('Luggage Storage')

df_hotel['Poolside Cafe'] = df_hotel['Pool Cafe'] + df_hotel['Poolside Bar']
feature_drop.append('Pool Cafe')
feature_drop.append('Poolside Bar')

df_hotel['Free Local Calls'] = df_hotel['Free Local Calls'] + df_hotel['Local Calls Free']
feature_drop.append('Local Calls Free')

# membuat feature baru seolah rename dan drop sisanya
df_hotel['Airport Transfer'] = df_hotel['Airport Transfer Available / Surcharge'] + df_hotel['Airport Transfer Free']
feature_drop.append('Airport Transfer Available / Surcharge')
feature_drop.append('Airport Transfer Free')

### Drop Hotel Feature (kuning)

In [62]:
# Jika error kemungkinan salah penulisan fitur, bisa jadi ada spasi seperti pada ' Cook Services'
feature_drop.append(' Cook Services')
feature_drop.append('Anti-Slip Ramps')
feature_drop.append('Audio - Visual Equipment')
feature_drop.append('Anti-Slip Ramps')
feature_drop.append('Audio - Visual Equipment')
feature_drop.append('Ayurvedic Facilities')
feature_drop.append('Boating')
feature_drop.append('Citrus Juicer')
feature_drop.append('Clock Radio')
feature_drop.append('Camel Ride')
feature_drop.append('Clothes Press')
feature_drop.append('Color TV')
feature_drop.append('Custom Duvets / Linen')
feature_drop.append('Courtesy Coach')
feature_drop.append('Dance Performances (on demand)')
feature_drop.append('Electronic Weighing Machine')
feature_drop.append('Fruit Basket')
feature_drop.append('Fire Exit Signs')
feature_drop.append('Handheld Showers')
feature_drop.append('Food Facility')
feature_drop.append('Horse Ride ( Chargeable ) ')
feature_drop.append('Hotel Trading License')
feature_drop.append('Luxurious Feather Mattresses')
feature_drop.append('Makeup / Shaving Mirror')
feature_drop.append('Moisturiser')
feature_drop.append('Kitchen available (home cook food on request)')
feature_drop.append('Laptop Available on Hire')
feature_drop.append('Pay Movies')
feature_drop.append('Pinup Boards')
feature_drop.append('Meditation Room')
feature_drop.append('Multi - Line Phone')
feature_drop.append('Sandwich Maker')
feature_drop.append('Shaving Kit /Dental Kit')
feature_drop.append('Shoe Shine')
feature_drop.append('Stationery Kit')
feature_drop.append('Sufficient Lighting')
feature_drop.append('Telephone in Toilet')
feature_drop.append('STD / ISD')
feature_drop.append('Safari')
feature_drop.append('Shopping Drop Facility (on fixed timings)')
feature_drop.append('Ski Storage')
feature_drop.append('Suitable For Children')
feature_drop.append('Transfers Available')
feature_drop.append('Trekking')
feature_drop.append('Veg / Non Veg Kitchens Separate ')
feature_drop.append('Village Visit ( Chargeable )')
feature_drop.append('Yoga Hall / Classes')

#df_hotel.drop(columns=feature_drop, inplace = True)

#print(feature_drop)

# jika sudah selesai tambahan inplace=True di akhir syntax

### Drop Duplicate Hotel/Room Feature (ungu)

In [63]:
# mirip seperti drop biasa (kuning)
# feature_drop.append('Laundry Service Available')
# feature_drop.append('Smoke Detector')
# feature_drop.append('Voicemail')
# feature_drop.append('Dry Cleaning')
# feature_drop.append('Locker Facility')

# update for handling duplicate feature (ungu)
df_hotel['Voicemail'] = df_hotel['Voicemail'] + df_hotel['Voicemail.1']
feature_drop.append('Voicemail.1')

df_hotel['Dry Cleaning'] = df_hotel['Dry Cleaning'] + df_hotel['Dry Cleaning.1']
feature_drop.append('Dry Cleaning.1')

df_hotel['Laundry Service Available'] = df_hotel['Laundry Service Available'] + df_hotel['Laundry Service Available.1']
feature_drop.append('Laundry Service Available.1')

df_hotel['Locker Facility'] = df_hotel['Locker Facility'] + df_hotel['Locker Facility.1']
feature_drop.append('Locker Facility.1')

df_hotel['Smoke Detector'] = df_hotel['Smoke Detector'] + df_hotel['Smoke Detector.1']
feature_drop.append('Smoke Detector.1')

df_hotel.drop(columns = feature_drop, inplace = True)

In [64]:
# mengembalikan/normalisasi nilai hasil merger ke 1/0
for i in range(len(df_hotel)):
    for col in df_hotel.columns[15:]:
        if df_hotel.loc[i, col] != 0:
            df_hotel.loc[i, col] = 1

In [65]:
# melakukan cek nilai maksimum dari seluruh fitur
df_hotel.iloc[:, 15:].max().max()

1

In [66]:
df_hotel.to_csv('DataHotel_ML_Filtered.csv')

## Add Hotel Images and Hotel Price 

In [11]:
# get non ML hotel dataset
df_hotel_nonML = pd.read_csv('https://raw.githubusercontent.com/zulfauzi92/Hotel_Recomendation_Model_Traveloka/main/Eksplorasi%20Data/Main%20Dataset/csv_final/Final_Dataset_Hotel_nonML.csv', index_col=[0])

In [12]:
df_hotel_nonML

hotel_id                                         hotel_name  \
0     H000001  Homewood Suites by Hilton RaleighDurham APRese...   
1     H000002  Four Points by Sheraton Orlando International ...   
2     H000003                Hampton Inn Pittsburgh/West Mifflin   
3     H000004                       JW Marriott Atlanta Buckhead   
4     H000005                               Best Western-rambler   
...       ...                                                ...   
1306  H001307                                      Cupertino Inn   
1307  H001308               Best Western Plus-concordville Hotel   
1308  H001309               Georgia Tech Hotel Conference Center   
1309  H001310          Best Western Plus Nashville Airport Hotel   
1310  H001311  Best Western Plus Dallas Hotel & Conference Ce...   

                                      hotel_description  \
0     The standard check-in time is 12:00 PM and the...   
1     The standard check-in time is 12:00 PM and the...   
2     The standard check-in time is 12:00 PM and the...   
3     The standard check-in time is 12:00 PM and the...   
4     The standard check-in time is 02:00 PM and the...   
...                                                 ...   
1306  The standard check-in time is 12:00 PM and the...   
1307  The standard check-in time is 12:00 PM and the...   
1308  The standard check-in time is 12:00 PM and the...   
1309  The standard check-in time is 10:00 AM and the...   
1310  The standard check-in time is 12:00 PM and the...   

                        address  \
0          4603 Central Park Dr   
1      5905 International Drive   
2      1550 Lebanon Church Road   
3            3300 Lenox Road NE   
4          457 US Highway 85 87   
...                         ...   
1306       10889 N De Anza Blvd   
1307          675 Conchester Rd   
1308           800 Spring St Nw   
1309      911 Airport Center Dr   
1310  8051 Lyndon B Johnson Fwy   

                                      point_of_interest        city state  \
0     ['Hadimba Temple', 'Naggar Village', 'Himalaya...      Durham    NC   
1     ['Sushant Lok', 'Sahara Mall', 'Amity Internat...     Orlando    FL   
2     ['The Mall', 'Tibetian Monastery', 'Vashisht H...  Pittsburgh    PA   
3     ['International Airport', 'Domestic Airport', ...     Atlanta    GA   
4     ['Others', 'K.L. Saini Stadium', 'BAPS Shri Sw...  Walsenburg    CO   
...                                                 ...         ...   ...   
1306                          ['Maharana Pratap Nagar']   Cupertino    CA   
1307  ['Gaffar Market', 'Gurudwara Rakabganj Sahib',...  Glen Mills    PA   
1308  ['The Naag (cobra) temple', 'Madhatop', 'Kud',...     Atlanta    GA   
1309                               ['Yasholaxmi Nagar']   Nashville    TN   
1310              ['Kota Airport', 'Station Road Area']      Dallas    TX   

     country  latitude  longitude property_type  hotel_star_rating  \
0         US      35.9      -78.8        Resort                  2   
1         US      28.5      -81.5   Guest House                  0   
2         US      40.3      -80.0       Cottage                  2   
3         US      33.8      -84.4         Hotel                  3   
4         US      37.7     -104.8      Homestay                  1   
...      ...       ...        ...           ...                ...   
1306      US      37.3     -122.0         Hotel                  2   
1307      US      39.9      -75.5         Hotel                  3   
1308      US      33.8      -84.4   Guest House                  0   
1309      US      36.1      -86.7         Hotel                  2   
1310      US      32.9      -96.8         Hotel                  2   

      site_review_rating                                   hotel_facilities  \
0                    4.2  ('Air / Rail Booking', 'Car Rental', 'Catering...   
1                    5.0  ('Baggage Room', 'Banquet Facilities', 'Busine...   
2                    5.0  ('Car Rental', 'Doctor on Cal

### hotel images

In [13]:
# create list of hotel images link
hotel_images = []
for i in range(1311):
    link = 'https://raw.githubusercontent.com/zulfauzi92/Hotel_Recomendation_Model_Traveloka/main/Hotel%20Images/image%20'
    link += str(i+1)
    link += '.jpg'
    hotel_images.append(link)
# add hotel_images link parameter to data non ML
df_hotel_nonML['hotel_images'] = hotel_images

In [15]:
df_hotel_nonML.head(4)

hotel_id                                         hotel_name  \
0  H000001  Homewood Suites by Hilton RaleighDurham APRese...   
1  H000002  Four Points by Sheraton Orlando International ...   
2  H000003                Hampton Inn Pittsburgh/West Mifflin   
3  H000004                       JW Marriott Atlanta Buckhead   

                                   hotel_description  \
0  The standard check-in time is 12:00 PM and the...   
1  The standard check-in time is 12:00 PM and the...   
2  The standard check-in time is 12:00 PM and the...   
3  The standard check-in time is 12:00 PM and the...   

                    address  \
0      4603 Central Park Dr   
1  5905 International Drive   
2  1550 Lebanon Church Road   
3        3300 Lenox Road NE   

                                   point_of_interest        city state  \
0  ['Hadimba Temple', 'Naggar Village', 'Himalaya...      Durham    NC   
1  ['Sushant Lok', 'Sahara Mall', 'Amity Internat...     Orlando    FL   
2  ['The Mall', 'Tibetian Monastery', 'Vashisht H...  Pittsburgh    PA   
3  ['International Airport', 'Domestic Airport', ...     Atlanta    GA   

  country  latitude  longitude property_type  hotel_star_rating  \
0      US      35.9      -78.8        Resort                  2   
1      US      28.5      -81.5   Guest House                  0   
2      US      40.3      -80.0       Cottage                  2   
3      US      33.8      -84.4         Hotel                  3   

   site_review_rating                                   hotel_facilities  \
0                 4.2  ('Air / Rail Booking', 'Car Rental', 'Catering...   
1                 5.0  ('Baggage Room', 'Banquet Facilities', 'Busine...   
2                 5.0  ('Car Rental', 'Doctor on Call', 'Dry Cleaning...   
3                 4.0  (' Drivers Rest Room', 'ATM / Banking', 'Adjoi...   

                                     room_facilities  \
0  ('Attached Bathroom', 'Bathrobes', 'Bathroom A...   
1  ('Air Conditioning', 'Attached Bathroom', 'Bat...   
2  ('Bathroom Amenities', 'Hot / Cold Running Wat...   
3  ('Air Conditioning', 'Bathrobes', 'Breakfast A...   

                                        hotel_images  
0  https://raw.githubusercontent.com/zulfauzi92/H...  
1  https://raw.githubusercontent.com/zulfauzi92/H...  
2  https://raw.githubusercontent.com/zulfauzi92/H...  
3  https://raw.githubusercontent.com/zulfauzi92/H...

### hotel price

In [16]:
# get ML hotel dataset for calculation hotel price
df_hotel_ML = pd.read_csv('https://raw.githubusercontent.com/zulfauzi92/Hotel_Recomendation_Model_Traveloka/main/Eksplorasi%20Data/Main%20Dataset/csv_final/Final_Dataset_Hotel_ML.csv', index_col=[0])

In [40]:
hotel_features = df_hotel_ML.iloc[:, 4:].copy()
hotel_features.head(4)

hotel_star_rating  site_review_rating  Beach Hut  BnB  Bungalow  Cottage  \
0                  2                 4.2          0    0         0        0   
1                  0                 5.0          0    0         0        0   
2                  2                 5.0          0    0         0        1   
3                  3                 4.0          0    0         0        0   

   Farm Stay  Guest House  Homestay  Hostel  Hotel  Houseboat  Lodge  Motel  \
0          0            0         0       0      0          0      0      0   
1          0            1         0       0      0          0      0      0   
2          0            0         0       0      0          0      0      0   
3          0            0         0       0      1          0      0      0   

   Palace  Resort  Service Apartment  Tent  Villa   Bike on Rent  \
0       0       1                  0     0      0              0   
1       0       0                  0     0      0              0   
2       0       0                  0     0      0              0   
3       0       0                  0     0      0              0   

    Drivers Rest Room  ATM / Banking  Access Via Exterior Corridors  \
0                   0              0                              0   
1                   0              0                              0   
2                   0              0                              0   
3                   1              1                              0   

   Adjoining Rooms  Air / Rail Booking  Air-Conditioning - Central  \
0                0                   1                           0   
1                0                   0                           0   
2                0                   0                           0   
3                1                   0                           1   

   Amphitheatre  Ayurvedic Facilities   Babysitting/ Child Care (surcharge)  \
0             0                      0                                    0   
1             0                      0                                    0   
2             0                      0                                    0   
3             0                      0                                    0   

   Badminttion Court  Baggage Room  Ballroom  Banquet Facilities  \
0                  0             0         0                   0   
1                  0             1         0                   1   
2                  0             0         0                   0   
3                  0             1         0                   1   

   Bar / Lounge   Barbecue Area  Basketball Court   Beauty Salon - on charge  \
0              0              0                  0                         0   
1              0              0                  0                         0   
2              0              0                  0                         0   
3              0              1                  0                         0   

   Billiards  Board Room  Bowling  Business Center  CCTV  Campfire / Bon Fire  \
0          0           0        0                0     0                    0   
1          0           0        0                1     0                    0   
2          0           0        0                0     0                    0   
3          0           1        0                1     1                    0   

   Car Rental  Casino  Catering  Cell Phone Rental  Children's Park  \
0           1       0         1                  0                1   
1           0       0         0                  0                0   
2           1       0         0                  0                0   
3           1       0         0                  0                0   

   City Shuttle Service  Clubhouse  Coffee Shop / Cafe  Computer Rental  \
0                     0          0                   0                0   
1                     0          0                   0                0   
2                     0          0    

In [51]:
ex = hotel_features.iloc[:25]

In [42]:
hotel_features.iloc[0, 4:].sum()

57.0

In [107]:
def calculate_hotel_price(data):
    output = []
    for i,item in data.iterrows():
        # set base price
        price = 50000
        price += item['hotel_star_rating']*100000
        price += item['site_review_rating']*30000
        # for each feature
        price += data.iloc[i, 4:].sum()*5000
        
        # add random modifier
        price += np.random.randint(-5,10)*5000
        
        output.append(price)
    return output
hotel_prices = calculate_hotel_price(hotel_features)

In [112]:
# price details
print('harga terendah : ',min(hotel_prices))
print('harga tertinggi : ',max(hotel_prices))
print('harga rata-rata : ',np.round(sum(hotel_prices)/len(hotel_prices)))

harga terendah :  85000.0
harga tertinggi :  1445000.0
harga rata-rata :  519111.0


In [115]:
# add hotel_images link parameter to data non ML
df_hotel_nonML['hotel_price'] = hotel_prices

In [116]:
df_hotel_nonML

hotel_id                                         hotel_name  \
0     H000001  Homewood Suites by Hilton RaleighDurham APRese...   
1     H000002  Four Points by Sheraton Orlando International ...   
2     H000003                Hampton Inn Pittsburgh/West Mifflin   
3     H000004                       JW Marriott Atlanta Buckhead   
4     H000005                               Best Western-rambler   
...       ...                                                ...   
1306  H001307                                      Cupertino Inn   
1307  H001308               Best Western Plus-concordville Hotel   
1308  H001309               Georgia Tech Hotel Conference Center   
1309  H001310          Best Western Plus Nashville Airport Hotel   
1310  H001311  Best Western Plus Dallas Hotel & Conference Ce...   

                                      hotel_description  \
0     The standard check-in time is 12:00 PM and the...   
1     The standard check-in time is 12:00 PM and the...   
2     The standard check-in time is 12:00 PM and the...   
3     The standard check-in time is 12:00 PM and the...   
4     The standard check-in time is 02:00 PM and the...   
...                                                 ...   
1306  The standard check-in time is 12:00 PM and the...   
1307  The standard check-in time is 12:00 PM and the...   
1308  The standard check-in time is 12:00 PM and the...   
1309  The standard check-in time is 10:00 AM and the...   
1310  The standard check-in time is 12:00 PM and the...   

                        address  \
0          4603 Central Park Dr   
1      5905 International Drive   
2      1550 Lebanon Church Road   
3            3300 Lenox Road NE   
4          457 US Highway 85 87   
...                         ...   
1306       10889 N De Anza Blvd   
1307          675 Conchester Rd   
1308           800 Spring St Nw   
1309      911 Airport Center Dr   
1310  8051 Lyndon B Johnson Fwy   

                                      point_of_interest        city state  \
0     ['Hadimba Temple', 'Naggar Village', 'Himalaya...      Durham    NC   
1     ['Sushant Lok', 'Sahara Mall', 'Amity Internat...     Orlando    FL   
2     ['The Mall', 'Tibetian Monastery', 'Vashisht H...  Pittsburgh    PA   
3     ['International Airport', 'Domestic Airport', ...     Atlanta    GA   
4     ['Others', 'K.L. Saini Stadium', 'BAPS Shri Sw...  Walsenburg    CO   
...                                                 ...         ...   ...   
1306                          ['Maharana Pratap Nagar']   Cupertino    CA   
1307  ['Gaffar Market', 'Gurudwara Rakabganj Sahib',...  Glen Mills    PA   
1308  ['The Naag (cobra) temple', 'Madhatop', 'Kud',...     Atlanta    GA   
1309                               ['Yasholaxmi Nagar']   Nashville    TN   
1310              ['Kota Airport', 'Station Road Area']      Dallas    TX   

     country  latitude  longitude property_type  hotel_star_rating  \
0         US      35.9      -78.8        Resort                  2   
1         US      28.5      -81.5   Guest House                  0   
2         US      40.3      -80.0       Cottage                  2   
3         US      33.8      -84.4         Hotel                  3   
4         US      37.7     -104.8      Homestay                  1   
...      ...       ...        ...           ...                ...   
1306      US      37.3     -122.0         Hotel                  2   
1307      US      39.9      -75.5         Hotel                  3   
1308      US      33.8      -84.4   Guest House                  0   
1309      US      36.1      -86.7         Hotel                  2   
1310      US      32.9      -96.8         Hotel                  2   

      site_review_rating                                   hotel_facilities  \
0                    4.2  ('Air / Rail Booking', 'Car Rental', 'Catering...   
1                    5.0  ('Baggage Room', 'Banquet Facilities', 'Busine...   
2                    5.0  ('Car Rental', 'Doctor on Cal